In [39]:
import numpy as np
import pandas as pd
import os, sys
import matplotlib.pyplot as plt
import sys
from PIL import Image, ImageOps
from pathlib import Path
from keras.preprocessing.image import img_to_array

from keras.datasets import mnist
import numpy as np
import numpy as np
import pandas as pd
import os, sys
import matplotlib.pyplot as plt
import sys
from PIL import Image, ImageOps
from pathlib import Path

from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
#from keras.optimizers import Adam

from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.utils import to_categorical

import tensorflow as tf

In [105]:
# importing and resizing image data 
path = "/Users/madelinewhitlow/Desktop/MIDS_W207/petfinder-pawpularity-score/train.csv"
df = pd.read_csv(path)
df['path'] = '/Users/madelinewhitlow/Desktop/MIDS_W207/petfinder-pawpularity-score/train/'+df['Id'] + '.jpg'
csv_path = "/Users/madelinewhitlow/Desktop/MIDS_W207/petfinder-pawpularity-score/train.csv"

df['img'] = df['path'].apply(lambda x: Image.open(x).convert("RGB").resize((20,20)))
#df['img'] = df['path'].apply(lambda x: ImageOps.grayscale(Image.open(x).resize((20,20))))


df['img_array'] = df['img'].apply(lambda x: img_to_array(x)/255)

#print(trainMdata.head())
labels=[]
imgArr=[]

for ind in df.index:
    img = df.img_array[0]
    imgArr.append(img)
    labels.append(df['Pawpularity'][ind])

labels = np.array(labels)
X_train=np.array(imgArr, dtype=float)
X_train.shape

(9912, 20, 20, 3)

In [106]:
shuffle = np.random.permutation(np.arange(X_train.shape[0]))

X_train, labels = X_train[shuffle], labels[shuffle]

print('data shape: ', X_train.shape)
print('label shape:', labels.shape)

# Set some variables to hold test, dev, and training data.
test_data, test_labels = X_train[8916:], labels[8916:]
dev_data, dev_labels = X_train[7920:8916], labels[7920:8916]
train_data, train_labels = X_train[:7921], labels[:7921]


test_labels1 = to_categorical(test_labels)
train_labels2 = to_categorical(train_labels)
train_labels2.shape


data shape:  (9912, 20, 20, 3)
label shape: (9912,)


(7921, 101)

In [116]:
shape = (20,20, 3)

# switch out activation to 'sigmoid'
# try different pixel amounts (100x100) (20x20) etc
#try binary crosstentropy vs categorical 
#adjust epochs
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten 
from keras.layers import MaxPooling2D, Dropout


model = Sequential()#add model layers
model.add(Conv2D(32, kernel_size=(5, 5),
                     activation='relu',
                     input_shape=shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
# add second convolutional layer with 20 filters
model.add(Conv2D(64, (5, 5), activation='relu'))
    
# add 2D pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))
    
# flatten data
model.add(Flatten())
    
# add a dense all-to-all relu layer
model.add(Dense(1024, activation='relu'))
    
# apply dropout with rate 0.5
model.add(Dropout(0.5))
    
# soft-max layer
model.add(Dense(101, activation='softmax'))
#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#train the model
model.fit(train_data,train_labels2, validation_data=(test_data, test_labels1), epochs=3)
# evaluate the model
score = model.evaluate(test_data, test_labels1, verbose=1)
# print performance
print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Epoch 1/3
248/248 [==============================] - 4s 12ms/step - loss: 0.0628 - accuracy: 0.0222 - val_loss: 0.0521 - val_accuracy: 0.0271
Epoch 2/3
248/248 [==============================] - 3s 12ms/step - loss: 0.0537 - accuracy: 0.0307 - val_loss: 0.0523 - val_accuracy: 0.0361
Epoch 3/3
32/32 [==============================] - 0s 4ms/step - loss: 0.0521 - accuracy: 0.0271

Test loss: 0.05208967998623848
Test accuracy: 0.027108434587717056


In [118]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn import metrics
from tensorflow.keras.regularizers import l2


In [100]:
train_data_temp = train_data.reshape(7921,30000)
test_data_temp = test_data.reshape(996, 30000)

model1 = KNeighborsClassifier(n_neighbors=1)
model1.fit(train_data_temp, train_labels)
test_predicted_labels = model1.predict(test_data_temp)
print("accuracy: ",metrics.accuracy_score(test_labels, test_predicted_labels))


accuracy:  0.008032128514056224


In [99]:
naive_bayes_model1 = BernoulliNB()
naive_bayes_model1.fit(train_data_temp, train_labels)
print ('Bernoulli Model accuracy: %3.4f' %naive_bayes_model1.score(test_data_temp, test_labels))

Bernoulli Model (Binary) accuracy: 0.0231


(7921,)

In [122]:
# model from kaggle page 
# https://www.kaggle.com/alexteboul/tutorial-part-3-cnn-image-modeling-1

# this accuracy seems ot be higher we could mess around with different filters/padding etc

inputs = tf.keras.Input(shape=(20,20,3))

#start off with x just being those inputs
x = inputs

x = tf.keras.layers.Conv2D(filters = 16, kernel_size = (7,7), strides = (2,2), padding='valid', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), padding='same', activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), strides = (2,2), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)


x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), strides = (2,2), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)

x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3),padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation = "relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)

output = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(inputs = inputs, outputs = output)

model.compile(
    loss = 'mse', 
    optimizer = 'Adam', 
    metrics = [tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"])

data_augmentation = ImageDataGenerator(
    rotation_range = 15, 
    zoom_range = 0.15,
    width_shift_range = 0.2, 
    height_shift_range = 0.2, 
    shear_range = 0.1,
    horizontal_flip = True, 
    fill_mode = "nearest")
history = model.fit(
    data_augmentation.flow(train_data,train_labels2,batch_size=32),
    validation_data = (test_data, test_labels1),
    steps_per_epoch = len(train_data) // 32,
    epochs = 3
)

Epoch 1/3
247/247 [==============================] - 9s 28ms/step - loss: 0.4700 - rmse: 0.5214 - mae: 0.3867 - mape: 376768896.0000 - val_loss: 0.2036 - val_rmse: 0.1063 - val_mae: 0.0386 - val_mape: 28447170.0000
Epoch 2/3
247/247 [==============================] - 6s 26ms/step - loss: 0.2426 - rmse: 0.2290 - mae: 0.1674 - mape: 157577712.0000 - val_loss: 0.1987 - val_rmse: 0.1048 - val_mae: 0.0343 - val_mape: 24181192.0000
Epoch 3/3
247/247 [==============================] - 7s 29ms/step - loss: 0.2055 - rmse: 0.1435 - mae: 0.0888 - mape: 79040712.0000 - val_loss: 0.1917 - val_rmse: 0.0992 - val_mae: 0.0253 - val_mape: 15542776.0000


In [123]:
score = model.evaluate(test_data, test_labels1, verbose=1)
print('Test accuracy:', score[1])

32/32 [==============================] - 0s 5ms/step - loss: 0.1917 - rmse: 0.0992 - mae: 0.0253 - mape: 15542776.0000
Test accuracy: 0.0991794690489769
